In [ ]:
#| default_exp vision_inference

In [ ]:
#| export
import numpy as np
from pathlib import Path
from torchio import Resize
from scipy.ndimage import label
from fastMONAI.vision_core import *
from fastMONAI.vision_augmentation import do_pad_or_crop
from skimage.morphology import remove_small_objects
from SimpleITK import DICOMOrient, GetArrayFromImage
from copy import copy

# Vision inference
>

In [ ]:
#| export
def _to_original_orientation(input_img, org_orientation):
    """Reorients the image to its original orientation."""
    
    orientation_itk = DICOMOrient(input_img, org_orientation)
    reoriented_array =  GetArrayFromImage(orientation_itk).transpose()
    
    return reoriented_array[None]

In [ ]:
#| export
def _do_resize(o, target_shape, image_interpolation='linear', 
               label_interpolation='nearest'):
    """
    Resample images so the output shape matches the given target shape.
    """

    resize = Resize(
        target_shape, 
        image_interpolation=image_interpolation, 
        label_interpolation=label_interpolation
    )
    
    return resize(o)

In [ ]:
#| export
def inference(learn_inf, reorder, resample, fn: (str, Path) = '',
              save_path: (str, Path) = None, org_img=None, input_img=None,
              org_size=None): 
    """Predict on new data using exported model."""         
    
    if None in [org_img, input_img, org_size]: 
        org_img, input_img, org_size = med_img_reader(fn, reorder, resample, 
                                                      only_tensor=False)
    else: 
        org_img, input_img = copy(org_img), copy(input_img)
    
    pred, *_ = learn_inf.predict(input_img.data)
    
    pred_mask = do_pad_or_crop(pred.float(), input_img.shape[1:], padding_mode=0, 
                               mask_name=None)
    input_img.set_data(pred_mask)
    
    input_img = _do_resize(input_img, org_size, image_interpolation='nearest')
    
    reoriented_array = _to_original_orientation(input_img.as_sitk(), 
                                                ('').join(org_img.orientation))
    
    org_img.set_data(reoriented_array)

    if save_path:
        save_fn = Path(save_path)/('pred_' + Path(fn).parts[-1])
        org_img.save(save_fn)
        return save_fn
    
    return org_img

## Post-processing

In [ ]:
#| export
def refine_binary_pred_mask(pred_mask, 
                            remove_size: (int, float) = None,
                            percentage: float = 0.2,
                            verbose: bool = False) -> np.ndarray:
    """Removes small objects from the predicted binary mask.

    Args:
        pred_mask: The predicted mask from which small objects are to be removed.
        remove_size: The size under which objects are considered 'small'.
        percentage: The percentage of the remove_size to be used as threshold. 
            Defaults to 0.2.
        verbose: If True, print the number of components. Defaults to False.

    Returns:
        The processed mask with small objects removed.
    """
                                
    labeled_mask, n_components = label(pred_mask)

    if verbose:
        print(n_components)

    if remove_size is None:
        sizes = np.bincount(labeled_mask.ravel())
        max_label = sizes[1:].argmax() + 1
        remove_size = sizes[max_label]

    small_objects_threshold = remove_size * percentage
    processed_mask = remove_small_objects(
        labeled_mask, min_size=small_objects_threshold)

    return np.where(processed_mask > 0, 1., 0.)